In [1]:
from abc import ABC, abstractmethod
from Constructor.objectTypes import Value, Frame as DataFrame
from Constructor.abstractNode import BaseNode
from Constructor.knotslinkers import BaseInputKnot, BaseOutputKnot

0.93125
40.0


In [1]:
from Constructor.abstractStrategyHub import *
import pandas

extractor = BaseStrategyExtractor()
hub = BaseStrategyHub()

extractor.create_parent_link(self_out_position=0, childNode=hub, child_in_position=1)
hub.create_parent_link(self_out_position=0, childNode=extractor, child_in_position=0)
[i['knot']._refreshChildrenKnots() for i in extractor.AllOutputs]
extractor.test_calculation()

print(hub.AllInputs[1]['knot'].getContent().return_batch)

[1, 2, 3, 4, 5]
